In [2]:
import pandas as pd
# import seaborn as sns
# from sklearn import tree
# import numpy as np


In [3]:
from subprocess import check_output
print(check_output(["ls", "data"]).decode("utf8"))

samplesubmission.csv
test.csv
train.csv
train_new.csv
train_ready.csv



In [4]:
train_data = pd.read_csv("data/train_ready.csv")

In [5]:
train_data.shape

(108129, 11)

In [6]:
train_data.head(2)

,project_id,goal,goal_log,country,currency,state_changed_at,backers_count,final_status,project_duration,creation_to_launch,launch_to_statechanged
0,kkst1451568084,20.0,1.301030,US,USD,2009-05-03 07:00:17,3,1,8,0,8
1,kkst1474482071,300.0,2.477121,US,USD,2009-05-16 00:00:18,2,0,16,0,16


In [7]:
train_copy = train_data.copy()

# Decision tree

In [8]:
from sklearn import tree
import numpy as np


In [9]:
classifier = tree.DecisionTreeClassifier()  # getting the tree

target = train_copy["final_status"].values
# features = train_copy[["goal", "backers_count", "creation_to_launch"]].values
features = train_copy[["goal", "goal_log", "backers_count", "creation_to_launch", "project_duration"]].values

In [10]:
classifier = classifier.fit(features, target)  # fitting/training the tree classifier

In [11]:
classifier

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

One way to quickly see the result of your decision tree is to see the importance of the features that are included. This is done by requesting the <b>.feature_importances_</b> attribute of your tree object. Another quick metric is the mean accuracy that you can compute using the <b>.score()</b> function with features_one and target as arguments.


In [12]:
# Look at the importance and score of the included features
print(classifier.feature_importances_)  # probably be calculating the inforamtion gain/entropy


[ 0.16813104  0.05362801  0.54825564  0.13929115  0.09069417]


The feature importances. The higher, the more important the feature. The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. It is also known as the <b>Gini importance</b> [R259].

In [13]:
classifier_two = tree.DecisionTreeClassifier(criterion='entropy')  # getting the tree
classifier_two = classifier_two.fit(features, target)  # fitting/training the tree classifier

In [14]:
print(classifier_two.feature_importances_)  # probably be calculating the inforamtion gain/entropy


[ 0.07042443  0.16456251  0.51244808  0.15226219  0.10030279]


# goal, backers_count, creation_to_launch
are the most important features - 

In [15]:
print(classifier.score(features, target))


0.993822193861


In [16]:
# import tensorflow as tf

# control overfitting

In [26]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical


<b>to_categorical</b>

to_categorical(y, num_classes=None)
Converts a class vector (integers) to binary class matrix.

E.g. for use with categorical_crossentropy.

Arguments

y: class vector to be converted into a matrix (integers from 0 to num_classes).

num_classes: total number of classes.

Returns - A binary matrix representation of the input.

# preparing test from train 80-20

<b>train_test_split()</b>

Split arrays or matrices into random train and test subsets
Quick utility that wraps input validation and next(ShuffleSplit().split(X, y)) and application to input data into a single call for splitting (and optionally subsampling) data in a oneliner.

In [18]:
#  classify label and features
#  make label to one_hot using to_categorical in keras
label = train_copy.final_status.values.tolist()  # a list of class labels i.e. final_status 0 failed, 1 succesful.
del train_copy['final_status']  # removing the class/label column, so we can extract the test examples


In [19]:
label[:10]  # extracted class labels.

[1, 0, 0, 1, 0, 0, 0, 1, 0, 0]

In [20]:
train_copy.head(1)


,project_id,goal,goal_log,country,currency,state_changed_at,backers_count,project_duration,creation_to_launch,launch_to_statechanged
0,kkst1451568084,20.0,1.30103,US,USD,2009-05-03 07:00:17,3,8,0,8


In [21]:
features = train_copy.values.tolist()  # extracting features as lists.


In [22]:
# type(features)
# len(features)
features[:2]


[['kkst1451568084',
  20.0,
  1.30102999566,
  'US',
  'USD',
  '2009-05-03 07:00:17',
  3,
  8,
  0,
  8],
 ['kkst1474482071',
  300.0,
  2.47712125472,
  'US',
  'USD',
  '2009-05-16 00:00:18',
  2,
  16,
  0,
  16]]

In [23]:
categorical_labels = to_categorical(label, 2)

In [24]:
type(categorical_labels)

numpy.ndarray

In [25]:
categorical_labels

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.]])

# train - test split

In [32]:
# split train and test data using scikit's train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, categorical_labels, test_size=0.2, random_state=40)


In [29]:
len(X_train)


86503

In [30]:
len(X_test)

21626

# A basic deep learning using keras - uses theano and tensorflow

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import advanced_activations  # all the available activation functions e.g. relu, 


In [ ]:
advanced_activations.

In [42]:
# the basic params for deep learning network
FEATURE_NUM = 10   # the input dimension - no of features == input layer nodes.
CLASSES = 2  # binary classification problem - successful 1 or failed 0 projects
# HIDDEN1_SIZE = 100
# HIDDEN2_SIZE = 50
HIDDEN1_SIZE = 20
HIDDEN2_SIZE = 10
MAX_RANGE = 100



These models have a number of methods in common:

<b>model.summary()</b>: prints a summary representation of your model

<b>model.get_config()</b>: returns a dictionary containing the configuration of the model. 

The model can be reinstantiated from its config via:
config = model.get_config()
model = Model.from_config(config)
 or, for Sequential:
model = Sequential.from_config(config)


<b>model.get_weights()</b>: returns a list of all weight tensors in the model, as Numpy arrays.

<b>model.set_weights(weights)</b>: sets the values of the weights of the model, from a list of Numpy arrays. The arrays in the list should have the same shape as those returned by get_weights().

In [35]:
model = Sequential()  # The Sequential model is a linear stack of layers.

<b>model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])</b>


You can also simply add layers via the <b>.add()</b> method:

model = Sequential

model.add(Dense(32, input_dim=784

model.add(Activation('relu'))

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                220       
Total params: 220
Trainable params: 220
Non-trainable params: 0
_________________________________________________________________


Both the following methods return - the representation that does not include the weights, only the architecture.

<b>model.to_json()</b>: returns a representation of the model as a JSON string. 

<b>model.to_yaml()</b>: returns a representation of the model as a YAML string. 

In [39]:
# model.to_json()

Specifying the input shape

The model needs to know what input shape it should expect. For this reason, the first layer in a Sequential model (and only the first, because following layers can do automatic shape inference) needs to receive information about its input shape. There are several possible ways to do this:

 - Pass an <b>input_shape</b> argument to the first layer. This is a shape tuple (a tuple of integers or None entries, where None indicates that any positive integer may be expected). In<b> input_shape</b>, the batch dimension is not included.

 - Some 2D layers, such as <b>Dense</b>, support the specification of their input shape via the argument <b>input_dim</b>, and some 3D temporal layers support the arguments <b>input_dim</b> and <b>input_length</b>.

 - If you ever need to specify a fixed batch size for your inputs (this is useful for stateful recurrent networks), you can pass a  batch_size argument to a layer. If you pass both <b>batch_size=32</b> and <b>input_shape=(6, 8)</b> to a layer, it will then expect every batch of inputs to have the batch shape <b>(32, 6, 8)</b>.

In [43]:
model.add(Dense(HIDDEN1_SIZE, input_dim=FEATURE_NUM, init='uniform'))

/home/nahmed/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, kernel_initializer="uniform", input_dim=10)`
  if __name__ == '__main__':


In [45]:
model.add(advanced_activations.ELU(alpha=1.0))
model.add(Dropout(0.6))
model.add(Dense(HIDDEN2_SIZE, init='uniform'))

/home/nahmed/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_initializer="uniform")`
  app.launch_new_instance()


In [46]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                220       
_________________________________________________________________
elu_1 (ELU)                  (None, 20)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
Total params: 430
Trainable params: 430
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.add(advanced_activations.ELU(alpha=1.0))
model.add(Dropout(0.6))
model.add(Dense(CLASSES, init='uniform', activation='softmax'))

/home/nahmed/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, activation="softmax", kernel_initializer="uniform")`
  app.launch_new_instance()


In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                220       
_________________________________________________________________
elu_1 (ELU)                  (None, 20)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
elu_2 (ELU)                  (None, 10)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 22        
Total para

# Training persistence - saving the minimized weights

<b>model.save_weights(filepath)</b>: saves the weights of the model as a HDF5 file.

<b>model.load_weights(filepath, by_name=False)</b>: loads the weights of the model from a HDF5 file (created by  save_weights). 

By default, the architecture is expected to be unchanged. 

To load weights into a different architecture (with some layers in common), use by_name=True to load only those layers with the same name.

In [49]:
checkpointer = ModelCheckpoint(filepath="/tmp/weights.hdf5",
                               verbose=1,
                               save_best_only=True)

In [50]:
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=False)

In [51]:
tensorboard.

In [52]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                220       
_________________________________________________________________
elu_1 (ELU)                  (None, 20)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
elu_2 (ELU)                  (None, 10)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 22        
Total para

# Training .....

In [54]:
model.fit(X_train, y_train,
          nb_epoch =MAX_RANGE,
          batch_size=1000,
          validation_data=(X_test, y_test),
          callbacks=[checkpointer, tensorboard])

/home/nahmed/anaconda2/lib/python2.7/site-packages/keras/models.py:851: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 86503 samples, validate on 21626 samples
Epoch 1/100


ValueError: could not convert string to float: kkst1213446772